In [1]:
import pandas as pd
import numpy as np

In [2]:
preprocessed_csv = pd.read_csv('Absenteeism_preprocessed.csv')

In [3]:
df = preprocessed_csv

In [4]:
df.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Month Value,Day of the Week
0,0,0,0,1,289,36,33,239.554,30,0,2,1,4,7,1
1,0,0,0,0,118,13,50,239.554,31,0,1,0,0,7,1
2,0,0,0,1,179,51,38,239.554,31,0,0,0,2,7,2
3,1,0,0,0,279,5,39,239.554,24,0,2,0,4,7,3
4,0,0,0,1,289,36,33,239.554,30,0,2,1,2,7,3


In [5]:
df['Absenteeism Time in Hours'].median()

3.0

In [6]:
targets = np.where(df['Absenteeism Time in Hours'] > 3,1,0)

In [7]:
targets

array([1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0,

In [8]:
df['Excessive Absenteeism'] = targets
df.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Month Value,Day of the Week,Excessive Absenteeism
0,0,0,0,1,289,36,33,239.554,30,0,2,1,4,7,1,1
1,0,0,0,0,118,13,50,239.554,31,0,1,0,0,7,1,0
2,0,0,0,1,179,51,38,239.554,31,0,0,0,2,7,2,0
3,1,0,0,0,279,5,39,239.554,24,0,2,0,4,7,3,1
4,0,0,0,1,289,36,33,239.554,30,0,2,1,2,7,3,0


In [9]:
df_with_targets = df.drop(['Absenteeism Time in Hours','Daily Work Load Average','Day of the Week',
                           'Distance to Work'],axis= 1)

In [10]:
df_with_targets.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Month Value,Excessive Absenteeism
0,0,0,0,1,289,33,30,0,2,1,7,1
1,0,0,0,0,118,50,31,0,1,0,7,0
2,0,0,0,1,179,38,31,0,0,0,7,0
3,1,0,0,0,279,39,24,0,2,0,7,1
4,0,0,0,1,289,33,30,0,2,1,7,0


In [11]:
unscaled_data = df_with_targets.iloc[:,:-1]

In [12]:
unscaled_data.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Month Value
0,0,0,0,1,289,33,30,0,2,1,7
1,0,0,0,0,118,50,31,0,1,0,7
2,0,0,0,1,179,38,31,0,0,0,7
3,1,0,0,0,279,39,24,0,2,0,7
4,0,0,0,1,289,33,30,0,2,1,7


In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin


In [14]:
class CustomScaler(BaseEstimator,TransformerMixin):

  def __init__(self,columns,copy=True,with_mean=True,with_std=True):
    self.scaler = StandardScaler(copy,with_mean,with_std)
    self.columns = columns
    self.mean_ = None
    self.var_ = None

  def fit(self, X, y=None):
    self.scaler.fit(X[self.columns], y)
    self.mean_ = np.mean(X[self.columns])
    self.var_ = np.var(X[self.columns]) 
    return self

  def transform(self, X, y=None, copy=None):
    init_col_order = X.columns
    X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
    X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
    return pd.concat([X_scaled,X_not_scaled], axis=1) [init_col_order]


In [15]:
unscaled_data.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Month Value'], dtype=object)

In [16]:
columns_to_scale = ['Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Month Value']

In [17]:
absenteeism_scaler = CustomScaler(columns_to_scale)

In [18]:
absenteeism_scaler.fit(unscaled_data)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CustomScaler(columns=['Transportation Expense', 'Age', 'Body Mass Index',
                      'Education', 'Children', 'Pets', 'Month Value'],
             copy=None, with_mean=None, with_std=None)

In [19]:
scaled_inputs = absenteeism_scaler.transform(unscaled_data)

In [20]:
scaled_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Month Value
0,0,0,0,1,1.005844,-0.536062,0.767431,-0.447980,0.880469,0.268487,0.182726
1,0,0,0,0,-1.574681,2.130803,1.002633,-0.447980,-0.019280,-0.589690,0.182726
2,0,0,0,1,-0.654143,0.248310,1.002633,-0.447980,-0.919030,-0.589690,0.182726
3,1,0,0,0,0.854936,0.405184,-0.643782,-0.447980,0.880469,-0.589690,0.182726
4,0,0,0,1,1.005844,-0.536062,0.767431,-0.447980,0.880469,0.268487,0.182726
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.654143,0.562059,-1.114186,2.232242,0.880469,-0.589690,-0.388293
696,1,0,0,0,0.040034,-1.320435,-0.643782,-0.447980,-0.019280,1.126663,-0.388293
697,1,0,0,0,1.624567,-1.320435,-0.408580,2.232242,-0.919030,-0.589690,-0.388293
698,0,0,0,1,0.190942,-0.692937,-0.408580,2.232242,-0.919030,-0.589690,-0.388293


In [21]:
from sklearn.model_selection import train_test_split

In [22]:
train_test_split(scaled_inputs,targets)

[     Reason_1  Reason_2  Reason_3  ...  Children      Pets  Month Value
 606         1         0         0  ... -0.919030 -0.589690    -1.244823
 14          0         0         0  ... -0.919030 -0.589690     0.182726
 630         1         0         0  ... -0.919030 -0.589690    -0.959313
 107         0         0         0  ...  2.679969 -0.589690     1.610276
 186         0         0         0  ...  0.880469 -0.589690    -0.673803
 ..        ...       ...       ...  ...       ...       ...          ...
 259         0         0         0  ...  0.880469  0.268487     0.468236
 114         0         0         1  ... -0.919030 -0.589690    -1.530333
 659         0         0         0  ... -0.919030 -0.589690    -0.673803
 136         0         0         0  ...  0.880469  0.268487    -1.530333
 604         0         0         0  ... -0.919030 -0.589690    -1.244823
 
 [525 rows x 11 columns],
      Reason_1  Reason_2  Reason_3  ...  Children      Pets  Month Value
 681         1         

In [23]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs,targets, train_size = 0.8, random_state = 20)

Modeling

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [25]:
reg = LogisticRegression()

In [26]:
reg.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [27]:
reg.score(x_train, y_train)

0.7732142857142857

In [28]:
reg.intercept_

array([-1.68218645])

In [29]:
reg.coef_

array([[ 2.80050879,  0.9550102 ,  3.11773817,  0.83794575,  0.60562747,
        -0.17008141,  0.27770724, -0.08455641,  0.34701569, -0.27797034,
         0.15857206]])

In [30]:
feature_names = unscaled_data.columns.values

In [31]:
summary_table = pd.DataFrame(columns=['Feature Names'], data= feature_names)

summary_table['Coefficient'] = np.transpose(reg.coef_)

summary_table

,Feature Names,Coefficient
0,Reason_1,2.800509
1,Reason_2,0.955010
2,Reason_3,3.117738
3,Reason_4,0.837946
4,Transportation Expense,0.605627
5,Age,-0.170081
6,Body Mass Index,0.277707
7,Education,-0.084556
8,Children,0.347016
9,Pets,-0.277970


In [32]:
summary_table.index = summary_table.index + 1

summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature Names,Coefficient
0,Intercept,-1.682186
1,Reason_1,2.800509
2,Reason_2,0.955010
3,Reason_3,3.117738
4,Reason_4,0.837946
5,Transportation Expense,0.605627
6,Age,-0.170081
7,Body Mass Index,0.277707
8,Education,-0.084556
9,Children,0.347016


In [33]:
summary_table['Odds Ratio'] = np.exp(summary_table.Coefficient)
summary_table

,Feature Names,Coefficient,Odds Ratio
0,Intercept,-1.682186,0.185967
1,Reason_1,2.800509,16.453016
2,Reason_2,0.955010,2.598697
3,Reason_3,3.117738,22.595215
4,Reason_4,0.837946,2.311613
5,Transportation Expense,0.605627,1.832402
6,Age,-0.170081,0.843596
7,Body Mass Index,0.277707,1.320100
8,Education,-0.084556,0.918920
9,Children,0.347016,1.414839


In [34]:
summary_table.sort_values('Odds Ratio', ascending=False)

,Feature Names,Coefficient,Odds Ratio
3,Reason_3,3.117738,22.595215
1,Reason_1,2.800509,16.453016
2,Reason_2,0.955010,2.598697
4,Reason_4,0.837946,2.311613
5,Transportation Expense,0.605627,1.832402
9,Children,0.347016,1.414839
7,Body Mass Index,0.277707,1.320100
11,Month Value,0.158572,1.171836
8,Education,-0.084556,0.918920
6,Age,-0.170081,0.843596


In [37]:
reg.score(x_test,y_test)

0.75

In [39]:
predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[0.71325648, 0.28674352],
       [0.58656313, 0.41343687],
       [0.44112564, 0.55887436],
       [0.78133175, 0.21866825],
       [0.08392008, 0.91607992],
       [0.3342291 , 0.6657709 ],
       [0.2993341 , 0.7006659 ],
       [0.13086501, 0.86913499],
       [0.78564217, 0.21435783],
       [0.7488237 , 0.2511763 ],
       [0.49371246, 0.50628754],
       [0.22445984, 0.77554016],
       [0.07098565, 0.92901435],
       [0.73468839, 0.26531161],
       [0.30807191, 0.69192809],
       [0.54979463, 0.45020537],
       [0.55013546, 0.44986454],
       [0.53890721, 0.46109279],
       [0.40098283, 0.59901717],
       [0.05349724, 0.94650276],
       [0.6997517 , 0.3002483 ],
       [0.78133175, 0.21866825],
       [0.41957064, 0.58042936],
       [0.41957064, 0.58042936],
       [0.24661288, 0.75338712],
       [0.74506925, 0.25493075],
       [0.50946316, 0.49053684],
       [0.85800128, 0.14199872],
       [0.20285591, 0.79714409],
       [0.78133175, 0.21866825],
       [0.

In [40]:
predicted_proba[:,1]

array([0.28674352, 0.41343687, 0.55887436, 0.21866825, 0.91607992,
       0.6657709 , 0.7006659 , 0.86913499, 0.21435783, 0.2511763 ,
       0.50628754, 0.77554016, 0.92901435, 0.26531161, 0.69192809,
       0.45020537, 0.44986454, 0.46109279, 0.59901717, 0.94650276,
       0.3002483 , 0.21866825, 0.58042936, 0.58042936, 0.75338712,
       0.25493075, 0.49053684, 0.14199872, 0.79714409, 0.21866825,
       0.37031375, 0.68010538, 0.68560828, 0.52889259, 0.21866825,
       0.53509531, 0.2220809 , 0.73802547, 0.4045697 , 0.60531851,
       0.21103167, 0.45055818, 0.2381166 , 0.39884737, 0.82740777,
       0.56944188, 0.69190691, 0.28674352, 0.22033975, 0.20359228,
       0.57434404, 0.32720156, 0.6657709 , 0.27050851, 0.83379804,
       0.43555357, 0.88424932, 0.23169099, 0.33361085, 0.34375056,
       0.69699469, 0.65562212, 0.29279038, 0.79300401, 0.20776315,
       0.26858753, 0.08760278, 0.2220809 , 0.73421318, 0.302073  ,
       0.2220809 , 0.29150074, 0.90416916, 0.46006255, 0.60162

In [41]:
import pickle

In [42]:
with open('model', 'wb') as file:
  pickle.dump(reg, file)

In [44]:
with open('custom_scaler', 'wb') as file:
  pickle.dump(absenteeism_scaler, file)